In [ ]:
!jt -t grade3 -fs 13 -nf opensans -nfs 13 -N -kl -cursw 5 -cursc r -cellw 1800 -T -ofs 13 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dates = pd.date_range('1/1/2000', periods=8)

In [ ]:
df = pd.DataFrame(np.random.randn(8, 4), index=dates, columns=['A', 'B', 'C', 'D'])

In [ ]:
 df = pd.read_csv("a_file")

In [ ]:
df

In [ ]:
train_df = df[df["Train/Test"] == "Train"]

In [ ]:
#train=train_df.iloc[:,1:] #removes "Test/Train column"

In [ ]:
train_df

In [ ]:
test_df = df[df["Train/Test"] == "Test"]

In [ ]:
test_df.reset_index(inplace=True, drop=True)


In [ ]:
#get author name
d1_df = df.iloc[2928]
d1_df[1] 

In [ ]:
#also get author name
d1_df = df[df.index == 2928]
author1 = d1_df['Author']
author1 = author1.values[0] 

allt ovan är bara läsa in df i två olika df's => test och train

In [ ]:
import torch
import random
from torch.autograd import Variable

In [ ]:
batch_size = 1
for i in range(batch_size):
    print(i)

In [ ]:
#df_filename = df.drop("Filename", axis=1) 

In [ ]:
samples = []
tensor_label_pairs = []
for i in range(batch_size):
    authors = list(set(df_filename["Author"]))
    seed_authors = random.choices(authors, k=2)
    data_train = df_filename[df_filename["Train/Test"] == "Train"]
    seed_entries_1 = (data_train[data_train["Author"] == seed_authors[0]]).sample(n=2, replace=True)
    #print(seed_entries_1.iloc[0])
    seed_entries_2 = (data_train[data_train["Author"] == seed_authors[1]]).sample(n=2, replace=True)
    if random.random() < 0.5:
        samples.append((seed_entries_1.iloc[0], seed_entries_1.iloc[1],0))
    else:
        samples.append((seed_entries_1.iloc[0], seed_entries_2.iloc[0],1))
for sample in samples:
    a = list(sample[0])[2:-2]
    #print(sample[0][1:-2])
   #print(sample[0].values[1:-1])
    b = list(sample[1])[2:-2]
    c = Variable(torch.Tensor((a+b)))
    label = torch.Tensor([sample[2]])
    print(sample[2])
    tensor_label_pairs.append((c,label))
print(tensor_label_pairs)


In [ ]:
authors = df.Author.unique().tolist()
print(len(authors))
first_author = authors.pop(random.randrange(0,(len(authors))))
print(len(authors))
print(first_author)
second_author = authors.pop(random.randrange(0,(len(authors))))
print(len(authors))
print(second_author)

In [ ]:
df1 = df[df["Author"].values == "bailey-s"]

row = np.random.choice(df.index.values)
#print(row)
sampled_df = df.iloc[row]
print(sampled_df[2:])

print(torch.from_numpy(df[df["Author"].values == first_author].sample(n=1).drop(["Train/Test", "Author"], axis=1).values))

In [ ]:
x = torch.randn(2, 3)
print(x)
y = torch.randn(2, 3)
print(y)
z = torch.randn(2, 3)
print(z)
torch.cat((x, y, z), 0)

In [ ]:

def sample_data(size, df):
    def compare_author(i1, i2):
        d1_df = df[df.index == i1]
        d2_df = df[df.index == i2]
        author1 = d1_df['0']
        a1 = author1.values.tolist()
        author2 = d2_df['0']
        a2 = author2.values.tolist()
        d1 = d1_df.values.tolist()
        d1 = d1[0][3:]
        d2 = d2_df.values.tolist()
        d2 = d2[0][3:]
        if a1 == a2:
            c = 1
        else:
            c = 0
        d = d1 + d2
        sample = (d, c)
        return sample

    samples = []
    counter1 = 0
    counter0 = 0
    while len(samples) <= size:
        index1 = random.choice(df.index)
        index2 = random.choice(df.index)
        while index1 == index2:
            index2 = random.choice(df.index)
        sample = compare_author(index1, index2)
        if sample[1] == 1:
            if counter1 <= size/2 + 1:
                samples.append(sample)
                counter1 += 1
        else:
            if counter0 <= size/2 + 1:
                samples.append(sample)
                counter0 += 1
    random.shuffle(samples)
    print(samples)

In [ ]:
sample_data(5, train_df)